In [6]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [7]:
data_dir = os.path.join(os.getcwd(), '../../pdiot-data/2023/')
s2100273 = os.path.join(data_dir, 's2100273')
s2100273_files = [file for file in os.listdir(s2100273) if "Respeck" in file and "clean" in file and file.endswith(".csv")]

In [8]:
cols = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'activity']
all_data = os.path.join(os.getcwd(), 'train_data.csv')
pd.DataFrame(columns=cols).to_csv(all_data, mode='w', header=True, index=False)

In [9]:
for file in s2100273_files:
    df = pd.read_csv(os.path.join(s2100273, file))
    act, subact = file.split('_')[2:4]
    df['activity'] = pd.Series(f"{act} {subact}", index=df.index)
    df = df[cols]
    df.to_csv(all_data, mode='a', header=False, index=False)

In [10]:
random_seed = 42
n_time_steps = 50
n_features = 6
step = 10
n_epochs = 40
batch_size = 64
learning_rate = 0.0025
l2_loss = 0.0015

segments = []
labels = []

In [11]:
df = pd.read_csv(all_data)

for i in range(0, len(df) - n_time_steps, step):
    xs = df['accel_x'].values[i: i + n_time_steps]
    ys = df['accel_y'].values[i: i + n_time_steps]
    zs = df['accel_z'].values[i: i + n_time_steps]
    gx = df['gyro_x'].values[i: i + n_time_steps]
    gy = df['gyro_y'].values[i: i + n_time_steps]
    gz = df['gyro_z'].values[i: i + n_time_steps]
    label = stats.mode(df['activity'][i: i + n_time_steps])[0][0]
    segments.append([xs, ys, zs, gx, gy, gz])
    labels.append(label)

/var/folders/1w/ty9_z0y12r72xymhb930rg9h0000gn/T/ipykernel_12342/366934639.py:10: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  label = stats.mode(df['activity'][i: i + n_time_steps])[0][0]
/var/folders/1w/ty9_z0y12r72xymhb930rg9h0000gn/T/ipykernel_12342/366934639.py:10: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  label = stats.mode(df['activity'][i: i + n_time_steps])[0][0]


In [12]:
np.array(segments).shape, np.array(labels).shape

((3338, 6, 50), (3338,))

In [13]:
reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)

labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

In [14]:
reshaped_segments.shape

(3338, 50, 6)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(reshaped_segments, labels, test_size=0.2, random_state=random_seed)

In [16]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((2670, 50, 6), (2670, 44), (668, 50, 6), (668, 44))

In [17]:
from tensorflow import keras
from keras import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten

In [18]:
model = Sequential()
model.add(LSTM(100, input_shape=(n_time_steps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(44, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               42800     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 44)                4444      
                                                                 
Total params: 57344 (224.00 KB)
Trainable params: 57344 (224.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.fit(x_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=1, validation_data=(x_test, y_test))

Epoch 1/40
42/42 [==============================] - 2s 29ms/step - loss: 3.6969 - accuracy: 0.0513 - val_loss: 3.4697 - val_accuracy: 0.0883
Epoch 2/40
42/42 [==============================] - 1s 25ms/step - loss: 3.1994 - accuracy: 0.0959 - val_loss: 2.8589 - val_accuracy: 0.1347
Epoch 3/40
42/42 [==============================] - 1s 24ms/step - loss: 2.7502 - accuracy: 0.1640 - val_loss: 2.4702 - val_accuracy: 0.2350
Epoch 4/40
42/42 [==============================] - 1s 25ms/step - loss: 2.3157 - accuracy: 0.2715 - val_loss: 1.9316 - val_accuracy: 0.3413
Epoch 5/40
42/42 [==============================] - 1s 24ms/step - loss: 1.8840 - accuracy: 0.3427 - val_loss: 1.6400 - val_accuracy: 0.4326
Epoch 6/40
42/42 [==============================] - 1s 25ms/step - loss: 1.6556 - accuracy: 0.4045 - val_loss: 1.3878 - val_accuracy: 0.5120
Epoch 7/40
42/42 [==============================] - 1s 25ms/step - loss: 1.4788 - accuracy: 0.4592 - val_loss: 1.2458 - val_accuracy: 0.5389
Epoch 8/40
42

In [21]:
model.evaluate(x_test, y_test)

21/21 [==============================] - 0s 6ms/step - loss: 0.3857 - accuracy: 0.8713


[0.385733962059021, 0.871257483959198]

In [105]:
# save model
model.save('model.h5')

/Users/nisiman/miniconda3/envs/pdiot/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# save tflite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/1w/ty9_z0y12r72xymhb930rg9h0000gn/T/tmpo1_a5wvh/assets


INFO:tensorflow:Assets written to: /var/folders/1w/ty9_z0y12r72xymhb930rg9h0000gn/T/tmpo1_a5wvh/assets
2023-10-16 11:55:22.019796: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-16 11:55:22.019814: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-16 11:55:22.020020: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/1w/ty9_z0y12r72xymhb930rg9h0000gn/T/tmpo1_a5wvh
2023-10-16 11:55:22.024856: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-10-16 11:55:22.024869: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/1w/ty9_z0y12r72xymhb930rg9h0000gn/T/tmpo1_a5wvh
2023-10-16 11:55:22.040941: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-10-16 11:55:22.093934: I tensorflow/cc/saved_model/loader.cc:215] Running initialization

In [55]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on one instance
interpreter.set_tensor(input_details[0]['index'], x_test[0:1])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[1.78167429e-11 5.00601340e-12 8.55394922e-12 3.20882833e-07
  1.08917445e-10 2.78121831e-07 4.69179984e-09 2.55874767e-07
  3.86373540e-06 2.10077880e-04 5.05074038e-10 3.95354582e-05
  7.89643964e-05 1.03025577e-07 1.12495329e-07 2.25265362e-09
  3.47531341e-07 1.19865762e-08 4.11333900e-07 2.72709570e-08
  3.17692747e-12 1.22776303e-10 3.10728630e-11 1.44265278e-09
  3.17306430e-08 1.72088876e-09 9.37989061e-11 3.31047689e-12
  8.53468598e-12 2.45209547e-11 4.86611836e-12 7.22961485e-01
  1.40677642e-07 1.18793420e-10 3.04056230e-05 2.14458734e-01
  6.08106367e-02 1.53306778e-09 1.10842451e-03 2.05766391e-08
  6.39469420e-07 2.09905465e-06 2.35933054e-04 5.71638557e-05]]


In [56]:
correct_predictions = 0
total_predictions = 0

for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]['index'], x_test[i:i+1])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    total_predictions += 1
    if np.argmax(output_data) == np.argmax(y_test[i]):
        correct_predictions += 1

accuracy = correct_predictions / total_predictions
print('Test accuracy:', accuracy)

Test accuracy: 0.8712574850299402
